# Random Forest

## Importamos las bibiliotecas que utilizaremos a lo largo del notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from preprocessing import prepararSetDeDatos
from preprocessing import arbolDeDecisionPreprocessing

## Importamos los datos y los procesamos

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')
X,y = prepararSetDeDatos(X,y)

In [ ]:
X_rf = arbolDeDecisionPreprocessing(X) #CREAR RF PREPROCESSING

## Funciones Auxiliares

In [ ]:
def obtenerMejoresParametros(datosPreprocesados):
    mejor_valor = 0
    mejor_profundidad = None
    mejor_cantidad_estimadores = None
    mejor_criterio = None
    y_array=np.array(y)
    for profundidad in [1,2,3,4,5,6,7,9,10]:
        for criterio in ["gini", "entropy"]:
            for cantidad_estimadores in [1,10,50,100,200]:
                kf = StratifiedKFold(n_splits=5)
                metricas = []
                for fold_idx, (train_index, test_index) in enumerate(kf.split(datosPreprocesados, y_array)):
                    rf = RandomForestClassifier(criterion=criterio, max_depth=profundidad, n_estimators=cantidad_estimadores)
                    rf.fit(datosPreprocesados[train_index], y_array[train_index].ravel())
                    predicciones = rf.predict(datosPreprocesados[test_index])
                    score_obtenida = roc_auc_score(y_array[test_index],predicciones)
                    metricas.append(score_obtenida)

                if np.mean(metricas) >= mejor_valor:
                    mejor_valor = np.mean(metricas)
                    mejor_profundidad = profundidad
                    mejor_criterio = criterio
                    mejor_cantidad_estimadores = cantidad_estimadores
            
    return mejor_valor, mejor_profundidad, mejor_criterio, mejor_cantidad_estimadores

In [ ]:
mejor_valor, mejor_profundidad, mejor_criterio, mejor_cantidad_estimadores = obtenerMejoresParametros(X_rf)

In [ ]:
print(mejor_profundidad)
print(mejor_criterio)
print(mejor_cantidad_estimadores)

### Dividimos el set de datos en sets de training y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_rf, y, test_size=0.25, random_state=0)

In [ ]:
rf = RandomForestClassifier(max_depth=mejor_profundidad, random_state=0, n_estimators = mejor_cantidad_estimadores, criterion=mejor_criterio)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

In [ ]:
fig, ax = plt.subplots(dpi =150)   
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, vmin = 0, yticklabels=["Volveria","No volveria"], xticklabels=["Volveria", "No Volveria"], ax=ax)
ax.set_title("Matriz de confusión de Random Forest")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

In [ ]:
fpr_rf_test,tpr_rf_test,thresholds_rf_test = roc_curve(y_test,rf.predict_proba(X_test)[:,1])

zero_test = np.argmin(np.abs(thresholds_rf_test))

plt.plot(fpr_rf_test,tpr_rf_test,label="ROC Curve Random Fores Test")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.plot(fpr_rf_test[zero_test],tpr_rf_test[zero_test],'o',markersize=10,label="threshold zero test",fillstyle="none",c="k",mew=2)

plt.legend(loc=4)
plt.show()

In [ ]:
auc_rf = roc_auc_score(y_test,rf.predict_proba(X_test)[:, 1])
print("AUC para Random Forest: {:.3f}".format(auc_rf))